<a href="https://colab.research.google.com/github/IntroComputationalPhysics-UNT/pivot-point-driven-pendulum-BraydenMittag/blob/main/StabilityHW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
# Import necessary libraries
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML


In [35]:
# Create pendulum animation
def create_pendulum_animation(t, x_pivot, y_pivot, x_lab, y_lab, l=1, show_traj=False, speed_factor=1.0):
    """
    Creates an animation of the pivot-driven pendulum.

    PARAMETERS:
    t : ndarray
        Time array.
    x_pivot : ndarray
        x-positions of the pivot over time.
    y_pivot : ndarray
        y-positions of the pivot over time.
    x_lab : ndarray
        x-positions of the pendulum bob in the lab frame over time.
    y_lab : ndarray
        y-positions of the pendulum bob in the lab frame over time.
    l : float, optional
        pendulum length (default is 1).
    show_traj : bool, optional
        Whether to show the trajectory of the pendulum bob (default is False).
    speed_factor : float, optional
        Factor to speed up or slow down the animation (default is 1.0).
    """
    fig, ax = plt.subplots(figsize=(4, 4)) # Increased figure size
    ax.set_xlim(np.min(x_pivot) - l, np.max(x_pivot) + l)
    ax.set_ylim(np.min(y_pivot) - l, np.max(y_pivot) + l)
    # ax.set_aspect('equal', adjustable='box')
    ax.set_aspect('equal')
    ax.set_xlabel('x')
    ax.set_ylabel('y')

    # Initialize plot elements
    pivot, = ax.plot([], [], 'o', color='black', markersize=8, label='Pivot')
    pendulum_arm, = ax.plot([], [], '-', color='black', lw=2, label='Pendulum Arm')
    pendulum_bob, = ax.plot([], [], 'o', color='red', markersize=12, label='Pendulum Bob')
    trajectory = None # Initialize trajectory to None

    if show_traj:
        trajectory, = ax.plot([], [], '-', color='gray', lw=1, alpha=0.5, label='Trajectory') # Add trajectory line

    def animate(i):
        # Update the positions of the plot elements
        pivot.set_data([x_pivot[i]], [y_pivot[i]]) # Pass as sequences
        pendulum_arm.set_data([x_pivot[i], x_lab[i]], [y_pivot[i], y_lab[i]])
        pendulum_bob.set_data([x_lab[i]], [y_lab[i]]) # Pass as sequences

        artists = [pivot, pendulum_arm, pendulum_bob] # List of artists to update

        if show_traj and trajectory:
             trajectory.set_data(x_lab[:i+1], y_lab[:i+1]) # Update trajectory data
             artists.append(trajectory) # Add trajectory to the list of artists

        return artists # Return all updated artists

    # Create the animation
    # Adjust the interval based on the average time step in t
    # This aims to make the animation speed consistent with the simulation time
    average_time_step = np.mean(np.diff(t))
    # Scale the interval by the speed_factor
    interval = average_time_step * 1000 * speed_factor # Convert to milliseconds and apply speed_factor.

    anim = FuncAnimation(fig, animate, frames=len(t), interval=interval, blit=True)
    plt.close(fig) # Close the initial figure to prevent it from displaying

    return anim

In [36]:
# Physical constants
g = 9.81  # m/s^2, acceleration due to gravity
l = 1.0   # m, length of the pendulum
m = 1.0   # kg, mass of the pendulum bob (cancels out in the EOM, but good for clarity)

# Drive parameters
omega_d = 2 * np.pi  # rad/s, driving frequency

# Equation of motion function for solve_ivp
def pendulum_eom(t, state):
    theta, theta_dot = state

    # Pivot motion and its derivatives
    x_p = -l * np.sin(omega_d * t)
    y_p = l * (1 - np.cos(omega_d * t))

    x_p_dot = -l * omega_d * np.cos(omega_d * t)
    y_p_dot = l * omega_d * np.sin(omega_d * t)

    x_p_ddot = l * omega_d**2 * np.sin(omega_d * t)
    y_p_ddot = l * omega_d**2 * np.cos(omega_d * t)

    # Equation of motion from the search results
    theta_ddot = (-g/l) * np.sin(theta) + (y_p_ddot/l) * np.sin(theta) + (x_p_ddot/l) * np.cos(theta)

    return [theta_dot, theta_ddot]

# Time parameters
t_span = (0, 20)  # Total simulation time
t_eval = np.linspace(t_span[0], t_span[1], 1000)

# Initial conditions (start at rest, vertically down)
initial_state = [0, 0]  # [theta_0, theta_dot_0]

# Numerical integration using solve_ivp
sol = solve_ivp(pendulum_eom, t_span, initial_state, t_eval=t_eval, method='RK45')

# Extract results
t = sol.t
theta = sol.y[0]
theta_dot = sol.y[1]

# Calculate positions for animation
x_pivot = -l * np.sin(omega_d * t)
y_pivot = l * (1 - np.cos(omega_d * t))

x_lab = x_pivot + l * np.sin(theta)
y_lab = y_pivot - l * np.cos(theta)


In [37]:
# Create and display the animation
animation = create_pendulum_animation(t, x_pivot, y_pivot, x_lab, y_lab, l=l, show_traj=True, speed_factor=1.0)
animation